## pip install
<big>    
* use $!pip$ $install$ in terminal is not installing under conda environment.
* probably work: $!conda$ $install$ for anaconda users.
* run in terminal (outside the conda environment):   
    * pip install pygame / pip install pymunk directly
    * windows users: Ctrl + r and type 'cmd'    
    * mac: https://macpaw.com/how-to/use-terminal-on-mac   
* Google Colab/JupyterHub wouldn't work. Have to run it locally.    
* specific question: ask me...?    


## Dead Kernel    
* I am confused as well, usually because of the memory allocation problem...I encountered them when dealing with 3,000,000 image data.   
* Segfault: 
```python
    def change_xy(self, new_xy):
        self.space.remove(self.space.bodies)
        self.__init__(new_xy)
```
* Other kernel problem?

In [7]:
"""
Template code for Assignment 7
(Based on modification of the pyramid demo from the box2d testbed in pymunk)
"""

import random
import time
import pygame
import numpy
from pygame.locals import *
from pygame.color import *
import pymunk
from pymunk import Vec2d
import pymunk.pygame_util

# Some general variables -- you don't need to change any of these
N_BLOCKS = 6 # How many blocks will fall?
BLOCK_SIZE = 20 # How big are the blocks?
deltaY     = 35 # How far spaced out vertically are they?
xSD        = 30.0 # What is the SD for their x-locations?
FPS = 30. # how many frames per second do we run?
BLOCK_MASS = 1.0
BLOCK_FRICTION = 1.0
FLOOR = 100
RUN_TIME = 10.0 # Time in seconds that we will run a simulation for
STEPS_PER_FRAME = 5.0 # Do not change this
WIDTH = 600 # Screen dimensions -- don't change
HEIGHT = 600

class BlockTower:
    # Implement a class to show/simulate blocks falling via pymunk
    # Note: this code has been modified from the pymunk pyramid demo

    def __init__(self, positions):
        # The intializer takes a list of x-positions for blocks; their height is set
        # by the code here.
        assert(len(positions)==N_BLOCKS) # can't give more than N_BLOCKS since we need to draw them

        self.positions = positions # store the positions of our blocks

        # Set up some pygame stuff
        self.running = True
        self.physics_running = False
        self.start_time = 0
        self.drawing = True
        self.w, self.h = WIDTH,HEIGHT
        self.screen = pygame.display.set_mode((self.w, self.h))
        self.clock = pygame.time.Clock()

        ### Init pymunk and create space
        self.space = pymunk.Space()
        self.space.gravity = (0.0, -900.0)
        self.space.sleep_time_threshold = 0.3

        self.floor = pymunk.Segment(self.space.static_body, (0, FLOOR), (self.w,FLOOR), 1.0)
        self.floor.friction = 1.0
        self.space.add(self.floor)
        print('init here')

        # Draw each block and add it to the physics
        for i in range(N_BLOCKS):
            points = [(-BLOCK_SIZE, -BLOCK_SIZE), (-BLOCK_SIZE, BLOCK_SIZE), (BLOCK_SIZE,BLOCK_SIZE), (BLOCK_SIZE, -BLOCK_SIZE)]
            moment = pymunk.moment_for_poly(BLOCK_MASS, points, (0,0))
            body = pymunk.Body(BLOCK_MASS, moment)
            xpos = self.positions[i]
            ypos = FLOOR + (2*i+1) * deltaY
            body.position = Vec2d(xpos,ypos)
            shape = pymunk.Poly(body, points)
            if(i == N_BLOCKS-1):     # color the top
                shape.color = (1,0,0,1)
                self.target_block = shape # store the top one we are tracking
            shape.friction = 1
            self.space.add(body,shape)
        print('after for loop')
        ### draw options for drawing
        self.draw_options = pymunk.pygame_util.DrawOptions(self.screen)
        

    def is_black_block_on_floor(self):
        # Returns true or false depending on whether the black block is on the bottom
        col = self.target_block.shapes_collide(self.floor) # this resturns a ContactPointSet
        print('is_black_block_on_floor')
        return len(col.points) > 0

    def run_person(self):
        # Show a window where people can predict yes/no (y/n) for whether the black block hits the bottom.
        # After they respond, they can observe the physics.
        # Rteturns their prediction and whether the black block actually hit the floor
        
        prediction = None # what people predicted?

        # Call this to run a single simulation with the given positions
        while self.running and (time.time() - self.start_time) < RUN_TIME or self.start_time==0:
            for event in pygame.event.get():
                if event.type == QUIT:
                    self.running = False
                elif event.type == KEYDOWN and event.key == K_ESCAPE:
                    self.running = False
                elif event.type == KEYDOWN and (event.key == K_y or event.key==K_n):  ## This detects a space press and starts simulating
                    prediction = (event.key == K_y)
                    self.physics_running = True
                    self.start_time = time.time()  # remember the time that physics started running

            if self.physics_running:
                self.space.step(1.0 / FPS / STEPS_PER_FRAME)  ## conveera frames per second to internal clock tics -- don't change!

            if self.drawing:
                self.draw()

            self.clock.tick(FPS) # don't let this loop run faster than FPS
        print('run_person')
        return (prediction, self.is_black_block_on_floor())

    def simulate(self):
        # Just run a simulation, returning whether after 10s the black block hits the floor
        for s in range(int(FPS*5*RUN_TIME)): # run for 10s
            self.space.step(1.0 / FPS / STEPS_PER_FRAME) # run this many steps
        print('simulate')
        return self.is_black_block_on_floor()

    def draw(self):
        ### This gets called to draw the scene

        ### Clear the screen
        self.screen.fill(THECOLORS["white"])

        ### Draw space  with our given options
        self.space.debug_draw(self.draw_options)

        ### All done, lets flip the display, which will cause it to be displayed
        pygame.display.flip()

    def change_xy(self, new_xy):
        self.space.remove(self.space.bodies)
        print('change_xy')
        self.__init__(new_xy)
########################################################################################################################
### Main code below
########################################################################################################################
for i in range(2):
    if __name__ == '__main__':
    
        # make some blocks at WIDTH/2 with a given SD
        positions = [numpy.random.normal(WIDTH/2, xSD) for _ in range(N_BLOCKS)]

        demo = BlockTower(positions)
        #print(demo.simulate())

        # Create a second one to show the blocks falling and get responses
        # (note: we CANNOT run demo.run_person because that one has alread run -- we have to make a new BlockTower object)
        #demo2 = BlockTower(positions)
        print(demo.run_person())
        pygame.quit()


init here
after for loop
run_person
is_black_block_on_floor
(True, False)
init here
after for loop
run_person
is_black_block_on_floor
(False, False)


This is how to use change_x_y:  
This would create instance every time.
```Python
for i in range(0, 1):
    for j in range(0, 10):
        position = pred.iloc[0, 0:6] + [numpy.random.normal(0, 10) for _ in range(6)]
        print([position])
        a = BlockTower(position).simulate()
        print(a)
```   
Use this instead:    
```Python
for i in range(0, 1):
    for j in range(0, 10):
        position =  pred.iloc[0, 0:6] + [numpy.random.normal(0, 10) for _ in range(6)]
        print([position])
        a = demo.change_xy(position)
        a.simulate()
pygame.quit()
```

In [6]:
for i in range(0, 1):
    for j in range(0, 2):
        position =  [numpy.random.normal(0, 10) for _ in range(6)]
        print([position])
        demo.change_xy(position)
        print(demo.simulate())
        
pygame.quit()

[[7.7712289679207105, -11.235607728370878, -7.378515569343014, -1.334483249573123, -4.040608496536788, 2.4315360498517564]]
change_xy
init here
after for loop
simulate
is_black_block_on_floor
True
[[-12.447049033783466, -12.245335848069718, -5.30195916528138, -1.2732392029885244, 4.234308001667531, 0.10440972965853187]]
change_xy
init here
after for loop
simulate
is_black_block_on_floor
True


## Sample code (from Aummul)   
* You could just use it.    
* Credit Aummul in your homework.

In [ ]:
import csv

with open('prediction.csv', 'w') as csvfile:
    wr = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    wr.writerows([["1","2","3","4","5","6","pred-truth"]])


In [ ]:
for i in range(0, 1):
    row = []
    row.append([numpy.random.normal(WIDTH/2, xSD) for _ in range(N_BLOCKS)])
    obj = BlockTower(row[0])
    print("type y/n")
    a = obj.run_person()
    print("1 run over")
    row[0].append(a)
    print(row)
    with open('prediction.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
        writer.writerows(row)
    

## Hints   
* You might want to look at the method run_person() to understand what the two outputs mean.   
* Please go through the whole notebook to know more about reading/writing to CSVs   
* I would recommend using the csv module for writing to a csv, and pandas to read the csv (more details below)    
* I would recommend writing each individual position separately (like the code block above) because it makes it easy to read the individual positions as integer later.

## How to do problem 2

<big>

**The y-axis is your subject (human) responses for those items.   
So for all the items whose model predictions fell between 10-20%, you compute what proportion of the time you said true/false on those items. That's the y-value. (Thus, each y-value will be based on a different number of points, because it will depend on how many items the model predicts that value for -- some might even be empty).**

after the 100 iterations for 200 items, we got a probability list for each item: call it $Prob$
a simple way to do that:
```Python
ten = []
twenty = []
thirty = []
forty = []
fifty = []
sixty = []
# keep creating lists...
for i in Prob:
    if i < 0.1:
        ten.append(i)
    elif i < 0.2:
        twenty.append(i)
#keep doing this if...elif...
```